# Visualising the traffic around Boston Airport

Aim of this notebook is to provide a codealong to be able to recreate the traffic from Square One Mall in Saugus, Massachusetts to Boston Logan Airport.

In [1]:
# Importing the necessary packages:
import pandas as pd
import numpy as np
import json
import requests

In [2]:
with open('response.json', 'r') as f:
    directions = json.load(f)

In [3]:
directions.keys()

dict_keys(['geocoded_waypoints', 'routes', 'status'])

In [4]:
route = directions['routes']

In [5]:
route

[{'bounds': {'northeast': {'lat': 42.4884743, 'lng': -71.0017644},
   'southwest': {'lat': 42.3648458, 'lng': -71.0298}},
  'copyrights': 'Map data ©2019',
  'legs': [{'distance': {'text': '10.6 mi', 'value': 17070},
    'duration': {'text': '23 mins', 'value': 1390},
    'duration_in_traffic': {'text': '34 mins', 'value': 2052},
    'end_address': 'Boston Logan International Airport 200 Terminal C Towards Gates C25-C36, Around the corner from the TSA Checkpoint, Boston, MA 02128, United States',
    'end_location': {'lat': 42.3678408, 'lng': -71.0188776},
    'start_address': '323 Broadway, Saugus, MA 01906, USA',
    'start_location': {'lat': 42.4882424, 'lng': -71.0177996},
    'steps': [{'distance': {'text': '85 ft', 'value': 26},
      'duration': {'text': '1 min', 'value': 5},
      'end_location': {'lat': 42.4884743, 'lng': -71.0177609},
      'html_instructions': 'Head <b>north</b>',
      'polyline': {'points': 'onibGftmpLm@G'},
      'start_location': {'lat': 42.4882424, 'lng

In [82]:
duration_in_traffic = None
root_steps = []
for legs in route:
    for steps in legs['legs']:
        for coordinates in steps['steps']:
            root_steps.append(coordinates['start_location'])
        root_steps.append(coordinates['end_location'])
    duration_in_traffic = steps['duration_in_traffic']['text']

#Chosen to only select the start coordinate of each step given that the end coordinate of step 0 
#is the start coordinate of step 1. The last coordinate is added at the end of the loop.

In [24]:
# Unpacking the root_steps list to retrieve only latitude and longitude data. Adding weights to represent different
# traffic congestions at each step of the root.

weighted_coordinates = [(root_step['lat'], root_step['lng'], np.random.randint(1,4) ) for root_step in root_steps]

In [25]:
weighted_coordinates_df = pd.DataFrame(weighted_coordinates,
                                       index=range(1, len(weighted_coordinates)+1),
                                       columns=['latitude', 'longitude', 'weight'])

In [26]:
weighted_coordinates_df

,latitude,longitude,weight
1,42.488242,-71.017800,2
2,42.488474,-71.017761,3
3,42.488347,-71.018141,2
4,42.487641,-71.017809,2
5,42.453675,-71.023521,3
6,42.432984,-71.017890,2
7,42.430901,-71.018533,3
8,42.421285,-71.005128,2
9,42.408770,-71.001764,3
10,42.376458,-71.028623,3


In [27]:
def get_keys(path):
    with open(path) as f:
        return json.load(f)
    
keys = get_keys("/Users/jjherranzsarrion/.secret/google_blog2_api.json")

In [30]:
import gmaps
import gmaps.datasets
%matplotlib inline

gmaps.configure(api_key=keys['api_key'])

locations = weighted_coordinates_df[['latitude', 'longitude']]
weights = weighted_coordinates_df['weight']
origin = (root_steps[0]['lat'], root_steps[0]['lng'])
destination = (root_steps[-1]['lat'], root_steps[-1]['lng'])

fig = gmaps.figure()
fig.add_layer(gmaps.directions_layer(origin, destination, show_markers=False, stroke_color='blue', stroke_opacity=0.4))
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights, opacity=0.8))
fig

Figure(layout=FigureLayout(height='420px'))

In [82]:
range(1, len(weighted_coordinates)+1)

range(1, 18)

In [52]:
weighted_coordinates_df['latitude'][len(weighted_coordinates_df)]

42.3678408

In [98]:
origin_destination = [{'name': ('Origin', 'The moment I realised the state of traffic'), 
                           'location': (weighted_coordinates_df['latitude'][1],
                                        weighted_coordinates_df['longitude'][1])},
                          {'name': 'Boston Logan Airport', 
                           'location': (weighted_coordinates_df['latitude'][len(weighted_coordinates_df)],
                                        weighted_coordinates_df['longitude'][len(weighted_coordinates_df)])}]
origin_destination

[{'name': ('Origin', 'The moment I realised the state of traffic'),
  'location': (42.4882424, -71.0177996)},
 {'name': 'Boston Logan Airport', 'location': (42.3678408, -71.0188776)}]

In [102]:
info_box_template = """
    <dl>
    <dt>{name[0]}</dt><dd>{name[1]}"""
markers_info = [info_box_template.format(**marker) for marker in origin_destination]
markers_info

['\n    <dl>\n    <dt>Origin</dt><dd>The moment I realised the state of traffic',
 '\n    <dl>\n    <dt>B</dt><dd>o']

In [103]:
def colour_coding_line(weighted_coordinates, duration_in_traffic):
    fig = gmaps.figure()

    origin_destination = [{'name': ('Origin', 'The moment I realised the state of traffic'), 
                           'location': (weighted_coordinates['latitude'][1],
                                        weighted_coordinates['longitude'][1]),
                           'ETA': duration_in_traffic},
                          {'name': ('Destination', 'Boston Logan Airport'), 
                           'location': (weighted_coordinates['latitude'][len(weighted_coordinates_df)],
                                        weighted_coordinates['longitude'][len(weighted_coordinates_df)]),
                           'ETA': duration_in_traffic}]

    markers_location = [marker['location'] for marker in origin_destination]
    info_box_template = """
    <dt>{name[0]}</dt><dd>{name[1]}<dd> 
    <dt>ETA:</dt>{ETA}<dd></p>
    """
    markers_info = [info_box_template.format(**marker) for marker in origin_destination]
    marker_layer = gmaps.marker_layer(markers_location, info_box_content=markers_info)
    fig.add_layer(marker_layer)
    
    for i in range(1, len(weighted_coordinates)):
        origin = (weighted_coordinates['latitude'][i], weighted_coordinates['longitude'][i])
        origin_weight = weighted_coordinates['weight'][i]
        destination = (weighted_coordinates['latitude'][i+1], weighted_coordinates['longitude'][i+1])
        destination_weight = weighted_coordinates['weight'][i+1]
        
        if origin_weight == 3:
            colour = 'red'
        elif origin_weight == 2:
            colour = (255,165,0) #tuple to represent orange
        else:
            colour='green'
        
        fig.add_layer(gmaps.directions_layer(origin, 
                                                 destination, 
                                                 show_markers=False, 
                                                 stroke_color=colour, 
                                                 stroke_opacity=0.8))
    
    return fig

In [104]:
colour_coding_line(weighted_coordinates_df, duration_in_traffic)

Figure(layout=FigureLayout(height='420px'))